In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from seq2seq import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle
from collections import Counter
import pickle
import torch
from sklearn.preprocessing import StandardScaler
from torch import Tensor

In [2]:
def set_seeds(seed):
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
# Convert a string that simulates a list to a real list
def convert_string_list(element):
    # Delete [] of the string
    element = element[0:len(element)]
    # Create a list that contains each code as e.g. 'A'
    ATC_list = list(element.split('; '))
    for index, code in enumerate(ATC_list):
        # Delete '' of the code
        ATC_list[index] = code[0:len(code)]
    return ATC_list

In [4]:
def multiplicate_rows(df):
    # Duplicate each compound the number of ATC codes associated to it, copying its SMILES in new rows
    new_rows = []
    
    for _, row in df.iterrows():
        atc_codes = row['ATC Codes']
        atc_codes_list = convert_string_list(atc_codes)
        
        if len(atc_codes_list) > 1:
            for code in atc_codes_list:
                if len(code) == 5:
                    new_row = row.copy()
                    new_row['ATC Codes'] = code
                    new_rows.append(new_row)
        else:
            if len(atc_codes_list[0]) == 5:
                new_rows.append(row)
    
    new_set = pd.DataFrame(new_rows)
    new_set = new_set.reset_index(drop=True)

    return new_set

def extract_descriptors(df):
    """
    Extract molecular descriptors from your dataset.
    You'll need to implement this based on your descriptor source.
    
    Returns FloatTensor of shape (n_molecules, descriptor_dimension)
    """
    descriptors = df.iloc[:, 2:-5].values
    # Convert to numpy for easier handling
    if isinstance(descriptors, torch.Tensor):
        desc_array = descriptors.numpy()
    else:
        desc_array = np.array(descriptors)
    # Replace infinite values with NaN first
    desc_array[np.isinf(desc_array)] = np.nan
    
    # Calculate median for each feature (column-wise)
    medians = np.nanmedian(desc_array, axis=0)
    
    # Replace NaN values with corresponding median
    for i in range(desc_array.shape[1]):
        mask = np.isnan(desc_array[:, i])
        desc_array[mask, i] = medians[i]
    return torch.tensor(desc_array, dtype=torch.float32)
    
# Create vocabularies
# Tokenize the data
def source(df):
    source = []
    for compound in df['Neutralized SMILES']:
        # A list containing each SMILES character separated
        source.append(list(compound))
    return source
def target(df):
    target = []
    for codes in df['ATC Codes']:  
        code = convert_string_list(codes) 
        # A list of lists, each one containing each ATC code character separated 
        for c in code:
            list_c = list(c)
            target.append(list_c)
    return target

In [5]:
seeds = [42, 123, 47899, 2025, 1, 20, 99, 1020, 345, 78] 
columns = [
    'Seed', 
    'Precision', 'Recall', 'F1',
    'Precision_level3', 'Recall_level3', 'F1_level3',
    'Precision_level2', 'Recall_level2', 'F1_level2',
    'Precision level 1', 'Precision level 2', 'Precision level 3', 'Precision level 4',
    'Recall level 1', 'Recall level 2', 'Recall level 3', 'Recall level 4',
    '#Compounds that have at least one match'
]
metrics_df = pd.DataFrame(columns=columns)

for seed in seeds:
    set_seeds(seed)

    train_set = pd.read_csv(f'Datasets/train_set{seed}.csv')
    test_set = pd.read_csv(f'Datasets/test_set{seed}.csv')
    val_set = pd.read_csv(f'Datasets/val_set{seed}.csv')
    
    new_train_set = multiplicate_rows(train_set)
    new_val_set = multiplicate_rows(val_set)
    new_test_set = multiplicate_rows(test_set)
    
    train_descriptors = extract_descriptors(new_train_set)
    test_descriptors = extract_descriptors(new_test_set)
    test_descriptors2 = extract_descriptors(test_set)
    val_descriptors = extract_descriptors(new_val_set)
    val_descriptors2 = extract_descriptors(val_set)
    
    scaler = StandardScaler()
    train_descriptors = torch.tensor(scaler.fit_transform(train_descriptors.numpy()), dtype=torch.float32)
    val_descriptors = torch.tensor(scaler.transform(val_descriptors.numpy()), dtype=torch.float32)
    test_descriptors = torch.tensor(scaler.transform(test_descriptors.numpy()), dtype=torch.float32)
    val_descriptors2 = torch.tensor(scaler.transform(val_descriptors2.numpy()), dtype=torch.float32)
    test_descriptors2 = torch.tensor(scaler.transform(test_descriptors2.numpy()), dtype=torch.float32)
    
    source_train = source(new_train_set)
    source_test = source(new_test_set)
    # Test set without duplicated compounds
    source_test2 = source(test_set)
    source_val = source(new_val_set)
    # Val set without duplicated compounds
    source_val2 = source(val_set)
    
    target_train = target(new_train_set)
    target_test = target(new_test_set)
    target_val = target(new_val_set)
    
    # An Index object represents a mapping from the vocabulary to integers (indices) to feed into the models
    source_index = index.Index(source_train)
    target_index = index.Index(target_train)
    
    # Create tensors
    X_train = source_index.text2tensor(source_train)
    y_train = target_index.text2tensor(target_train)
    X_val = source_index.text2tensor(source_val)
    X_val2 = source_index.text2tensor(source_val2)
    y_val = target_index.text2tensor(target_val)     
    X_test = source_index.text2tensor(source_test)
    X_test2 = source_index.text2tensor(source_test2)
    y_test = target_index.text2tensor(target_test)

    if torch.cuda.is_available():
        X_train = X_train.to("cuda")
        y_train = y_train.to("cuda")
        train_descriptors = train_descriptors.to("cuda") 
        test_descriptors = test_descriptors.to("cuda")
        test_descriptors2 = test_descriptors2.to("cuda")
        val_descriptors = val_descriptors.to("cuda")
        val_descriptors2 = val_descriptors2.to("cuda")
        X_val = X_val.to("cuda")
        X_val2 = X_val2.to("cuda")
        y_val = y_val.to("cuda")
        X_test= X_test.to("cuda")
        y_test = y_test.to("cuda")
        X_test2 = X_test2.to("cuda")

    model = multimodal_models.MultimodalTransformer(
                source_index, 
                target_index,
                max_sequence_length = 800,
                embedding_dimension = 128,
                descriptors_dimension=train_descriptors.shape[1],
                feedforward_dimension = 256,
                encoder_layers = 4,
                decoder_layers = 3,
                attention_heads = 4,
                activation = "relu",
                dropout = 0.0)   
    model.to("cuda")
    q = model.fit(X_train, 
            train_descriptors,
            y_train,
            X_val, 
            val_descriptors,
            y_val, 
            batch_size = 32, 
            epochs = 150, 
            learning_rate = 0.0001, 
            weight_decay = 1e-05,
            progress_bar = 0, 
            save_path = None)
    model.load_state_dict(torch.load("best_multimodalmodel.pth", weights_only=True))
    loss, error_rate = model.evaluate(X_test, test_descriptors, y_test, batch_size = 32) 

    predictions, log_probabilities = search_algorithms.multimodal_beam_search(
        model, 
        X_test2, 
        test_descriptors2,
        predictions = 6, # max length of the predicted sequence
        beam_width = 10,
        batch_size = 32, 
        progress_bar = 0
    )
    output_beam = [target_index.tensor2text(p) for p in predictions]
    predictions_clean = []
    for preds in output_beam:
        interm = []
        for pred in preds:
            clean_pred = pred.replace('<START>', '').replace('<END>', '')
            if len(clean_pred) == 5:
                interm.append(clean_pred)
            if len(interm) == 3:
                break
        predictions_clean.append(interm)
    predictions_clean_level3 = []
    for preds in output_beam:
        interm = []
        for pred in preds:
            clean_pred = pred.replace('<START>', '').replace('<END>', '')
            pred_3 = clean_pred[0:4]
            if len(pred_3) == 4 and pred_3 not in interm:
                interm.append(pred_3)
        predictions_clean_level3.append(interm[0:3])
    predictions_clean_level2 = []
    for preds in output_beam:
        interm = []
        for pred in preds:
            clean_pred = pred.replace('<START>', '').replace('<END>', '')
            pred_2 = clean_pred[0:3]
            if len(pred_2) == 3 and pred_2 not in interm:
                interm.append(pred_2)
        predictions_clean_level2.append(interm[0:3])
    precision_1, precision_2, precision_3, precision_4 = defined_metrics.precision(predictions_clean, f'Datasets/test_set{seed}.csv', 'ATC Codes')
    recall_1, recall_2, recall_3, recall_4, counter_compound_match = defined_metrics.recall(predictions_clean, f'Datasets/test_set{seed}.csv', 'ATC Codes')
    precisions, recalls, f1s = defined_metrics.complete_metrics(predictions_clean, f'Datasets/test_set{seed}.csv', 'ATC Codes', 3)
    precisions_level3, recalls_level3, f1s_level3 = defined_metrics.complete_metrics_level3(predictions_clean_level3, f'Datasets/test_set{seed}.csv', 'ATC Codes', 3)
    precisions_level2, recalls_level2, f1s_level2 = defined_metrics.complete_metrics_level2(predictions_clean_level2, f'Datasets/test_set{seed}.csv', 'ATC Codes', 3)
    precisions_average = sum(precisions)/len(precisions)
    recalls_average = sum(recalls)/len(recalls)
    f1s_average = sum(f1s)/len(f1s)

    precisions_average_level3 = sum(precisions_level3)/len(precisions_level3)
    recalls_average_level3 = sum(recalls_level3)/len(recalls_level3)
    f1s_average_level3 = sum(f1s_level3)/len(f1s_level3)

    precisions_average_level2 = sum(precisions_level2)/len(precisions_level2)
    recalls_average_level2 = sum(recalls_level2)/len(recalls_level2)
    f1s_average_level2 = sum(f1s_level2)/len(f1s_level2)
    
    metrics = {
        'Precision': precisions_average, 
        'Recall': recalls_average,
        'F1': f1s_average,
        'Precision_level3': precisions_average_level3, 
        'Recall_level3': recalls_average_level3,
        'F1_level3': f1s_average_level3,
        'Precision_level2': precisions_average_level2, 
        'Recall_level2': recalls_average_level2,
        'F1_level2': f1s_average_level2,
        'Precision level 1': precision_1,
        'Precision level 2': precision_2,
        'Precision level 3': precision_3,
        'Precision level 4': precision_4,
        'Recall level 1': recall_1,
        'Recall level 2': recall_2,
        'Recall level 3': recall_3,
        'Recall level 4': recall_4,
        '#Compounds that have at least one match': counter_compound_match
    }
    
    row = {
        'Seed': seed,
        **metrics
    }
    
    metrics_df = pd.concat([metrics_df, pd.DataFrame([row])], ignore_index=True)

    torch.cuda.empty_cache()

metrics_df.to_csv("multimodaltransformer_metrics.csv", index=False)
print("Mean:", metrics_df.mean(numeric_only=True))
print("Std:", metrics_df.std(numeric_only=True))

C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Model: Seq2Seq Multimodal Transformer
Source index: <Seq2Seq Index with 43 items>
Target index: <Seq2Seq Index with 34 items>
Max sequence length: 800
Embedding dimension: 128
Descriptors dimension: 1136
Feedforward dimension: 256
Encoder layers: 4
Decoder layers: 3
Attention heads: 4
Activation: relu
Dropout: 0.0
Trainable parameters: 1,405,346

Training started
X_train.shape: torch.Size([3057, 702])
Y_train.shape: torch.Size([3057, 7])
X_dev.shape: torch.Size([546, 337])
Y_dev.shape: torch.Size([546, 7])
Epochs: 150
Learning rate: 0.0001
Weight decay: 1e-05
Epoch | Train                 | Development           | Minutes
      | Loss     | Error Rate | Loss     | Error Rate |
---------------------------------------------------------------
    1 |   1.9469 |     50.273 |   1.4584 |     43.651 |     0.2
    2 |   1.3092 |     39.576 |   1.2756 |     38.828 |     0.4
    3 |   1.1661 |     35.863 |   1.1842 |     36.142 |     0.6
    4 |   1.0628 |     33.110 |   1.1212 |     34.860 |   

C:\Users\trini\AppData\Local\Temp\ipykernel_11092\3793583644.py:187: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, pd.DataFrame([row])], ignore_index=True)
C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Model: Seq2Seq Multimodal Transformer
Source index: <Seq2Seq Index with 45 items>
Target index: <Seq2Seq Index with 34 items>
Max sequence length: 800
Embedding dimension: 128
Descriptors dimension: 1136
Feedforward dimension: 256
Encoder layers: 4
Decoder layers: 3
Attention heads: 4
Activation: relu
Dropout: 0.0
Trainable parameters: 1,405,602

Training started
X_train.shape: torch.Size([3048, 649])
Y_train.shape: torch.Size([3048, 7])
X_dev.shape: torch.Size([541, 337])
Y_dev.shape: torch.Size([541, 7])
Epochs: 150
Learning rate: 0.0001
Weight decay: 1e-05
Epoch | Train                 | Development           | Minutes
      | Loss     | Error Rate | Loss     | Error Rate |
---------------------------------------------------------------
    1 |   1.9461 |     50.421 |   1.4390 |     43.500 |     0.2
    2 |   1.3260 |     40.890 |   1.2568 |     38.201 |     0.3
    3 |   1.1848 |     36.822 |   1.1457 |     35.336 |     0.5
    4 |   1.0788 |     33.547 |   1.0940 |     33.672 |   

C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


...

Model: Seq2Seq Multimodal Transformer
Source index: <Seq2Seq Index with 44 items>
Target index: <Seq2Seq Index with 34 items>
Max sequence length: 800
Embedding dimension: 128
Descriptors dimension: 1136
Feedforward dimension: 256
Encoder layers: 4
Decoder layers: 3
Attention heads: 4
Activation: relu
Dropout: 0.0
Trainable parameters: 1,405,474

Training started
X_train.shape: torch.Size([3052, 702])
Y_train.shape: torch.Size([3052, 7])
X_dev.shape: torch.Size([533, 266])
Y_dev.shape: torch.Size([533, 7])
Epochs: 150
Learning rate: 0.0001
Weight decay: 1e-05
Epoch | Train                 | Development           | Minutes
      | Loss     | Error Rate | Loss     | Error Rate |
---------------------------------------------------------------
    1 |   2.0189 |     53.866 |   1.4124 |     42.933 |     0.2
    2 |   1.3188 |     40.312 |   1.2509 |     38.962 |     0.4
    3 |   1.1852 |     36.730 |   1.1675 |     35.679 |     0.6
    4 |   1.0902 |     34.207 |   1.1139 |     34.709 |   

C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Model: Seq2Seq Multimodal Transformer
Source index: <Seq2Seq Index with 43 items>
Target index: <Seq2Seq Index with 34 items>
Max sequence length: 800
Embedding dimension: 128
Descriptors dimension: 1136
Feedforward dimension: 256
Encoder layers: 4
Decoder layers: 3
Attention heads: 4
Activation: relu
Dropout: 0.0
Trainable parameters: 1,405,346

Training started
X_train.shape: torch.Size([3085, 702])
Y_train.shape: torch.Size([3085, 7])
X_dev.shape: torch.Size([537, 649])
Y_dev.shape: torch.Size([537, 7])
Epochs: 150
Learning rate: 0.0001
Weight decay: 1e-05
Epoch | Train                 | Development           | Minutes
      | Loss     | Error Rate | Loss     | Error Rate |
---------------------------------------------------------------
    1 |   2.0134 |     52.901 |   1.4515 |     43.234 |     0.2
    2 |   1.3298 |     40.529 |   1.2714 |     39.665 |     0.4
    3 |   1.1840 |     36.926 |   1.1858 |     36.840 |     0.6
    4 |   1.0732 |     33.668 |   1.1203 |     35.506 |   

C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Model: Seq2Seq Multimodal Transformer
Source index: <Seq2Seq Index with 46 items>
Target index: <Seq2Seq Index with 34 items>
Max sequence length: 800
Embedding dimension: 128
Descriptors dimension: 1136
Feedforward dimension: 256
Encoder layers: 4
Decoder layers: 3
Attention heads: 4
Activation: relu
Dropout: 0.0
Trainable parameters: 1,405,730

Training started
X_train.shape: torch.Size([3057, 702])
Y_train.shape: torch.Size([3057, 7])
X_dev.shape: torch.Size([521, 307])
Y_dev.shape: torch.Size([521, 7])
Epochs: 150
Learning rate: 0.0001
Weight decay: 1e-05
Epoch | Train                 | Development           | Minutes
      | Loss     | Error Rate | Loss     | Error Rate |
---------------------------------------------------------------
    1 |   1.9756 |     51.690 |   1.4530 |     43.282 |     0.2
    2 |   1.3371 |     41.277 |   1.2817 |     39.571 |     0.4
    3 |   1.1840 |     36.136 |   1.1811 |     37.172 |     0.6
    4 |   1.0765 |     33.421 |   1.1031 |     35.317 |   